<a href="https://colab.research.google.com/github/sipocz/_earthQuake/blob/main/_EarthQuake_outlier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [260]:
from sklearn.ensemble import IsolationForest
import numpy as np



In [261]:
#EarthQuake
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import sweetviz as sw

#!pip install sweetviz
#import sweetviz as sw
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [262]:
df=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/features_a.csv")
df_clasters=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/train_labels.csv")
df_testvalues=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_a.csv")
#official_testvalues=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_values.csv")


In [263]:
numx=260601
#numx=50000
Y=df_clasters[["damage_grade"]]
X=df
scaler=MinMaxScaler()
Xt=X[:numx]
Xt=scaler.fit_transform(Xt)
Xt=Xt[:numx]
Y=Y[["damage_grade"]]
Y=Y[:numx]
X_train, X_test, y_train, y_test = train_test_split(Xt, Y, random_state=2,test_size=0.0001)
df_buildings=df_testvalues[["building_id"]]
heads=df.columns
print(len(heads))
print(len(df_testvalues.columns))
for i in df_testvalues.columns:
    if i not in heads:
        df_testvalues.drop(columns=[i], inplace=True)

print(len(df_testvalues.columns))
print(len(X_train))
print(X_train)

57
78
57
260574
[[0.57266692 0.26666667 0.70707779 ... 0.         0.         1.        ]
 [0.9415848  0.86666667 0.71688858 ... 0.         0.         1.        ]
 [0.12814275 0.13333333 0.41205326 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]


In [264]:
# inlier és outlier bombázás
clf = IsolationForest(n_estimators=3, warm_start=True, max_features=12)
clf.fit(X_train)  # fit 10 trees  
#clf.set_params()  # add 10 more trees  
#clf.fit(X)  # fit the added trees 

IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=12, max_samples='auto', n_estimators=3,
                n_jobs=None, random_state=None, verbose=0, warm_start=True)

In [265]:
X_train_predict=clf.predict(X_train)
print(len(X_train))
print(X_train)

260574
[[0.57266692 0.26666667 0.70707779 ... 0.         0.         1.        ]
 [0.9415848  0.86666667 0.71688858 ... 0.         0.         1.        ]
 [0.12814275 0.13333333 0.41205326 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]


In [266]:
def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")
outlierStatistic(X_train_predict)

[1 1 1 ... 1 1 1]
A összes (260574 darabból 54060 darab outlier van. Az  20.7 %.)


In [267]:
# az official test vizsgálata outlierekre nézve:
outlier_official_predikt=clf.predict(df_testvalues)

In [268]:
outlierStatistic(outlier_official_predikt)

[-1 -1 -1 ... -1 -1 -1]
A összes (86868 darabból 86493 darab outlier van. Az  99.6 %.)


In [269]:
print(list(y_train[["damage_grade"]].index)[0])

149237


In [270]:
print(X_train[1])

[0.9415848  0.86666667 0.71688858 0.125      0.01507538 0.06060606
 0.1        0.         1.         0.         0.         0.
 0.         1.         0.         0.         0.         0.
 0.11111111 0.         0.         0.         0.         0.
 0.         0.         0.         1.         0.         0.
 1.         0.         0.         0.         0.         1.
 0.         0.         0.         1.         0.         0.
 0.         0.         1.         0.         1.         0.
 0.         0.         1.         0.         0.         0.
 0.         0.         1.        ]


In [271]:

y_train["damage_grade"].values

array([3, 2, 2, ..., 2, 2, 3])

In [272]:


X_train_inliers=[X_train[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
X_train_outliers=[X_train[index] for index,i in enumerate(X_train_predict) if i==-1 ]
tmp=list(y_train["damage_grade"].values)
y_train_inliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
y_train_outliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==-1 ]



In [273]:
dX1=pd.DataFrame (X_train_inliers)
dY1=pd.DataFrame (y_train_inliers)
DtX=pd.DataFrame(X_test)

In [274]:
# KNN inlier modell

from xgboost import XGBClassifier  # 72.09
# max_depth=10 : 72.79857561664441



knn_inlier = XGBClassifier(verbosity=3,max_depth = 35)

[21:54:21] ======== Monitor: Learner ========
[21:54:21] GetGradient: 3.07378s, 100 calls @ 30737us
[21:54:21] PredictRaw: 0.218167s, 100 calls @ 2181us
[21:54:21] UpdateOneIter: 1448.27s, 100 calls @ 14482689us
[21:54:21] ======== Monitor: GBTree ========
[21:54:21] BoostNewTrees: 1404.76s, 100 calls @ 14047589us
[21:54:21] CommitModel: 40.2167s, 100 calls @ 402166us


In [ ]:
print("Fit Start--")
knn_inlier.fit(dX1, dY1)
print("Fit End--")

Fit Start--


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[21:54:27] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 13908 extra nodes, 0 pruned nodes, max_depth=34
[21:54:32] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46244 extra nodes, 0 pruned nodes, max_depth=35
[21:54:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44148 extra nodes, 0 pruned nodes, max_depth=35
[21:54:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14618 extra nodes, 0 pruned nodes, max_depth=35
[21:54:47] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 47960 extra nodes, 0 pruned nodes, max_depth=35
[21:54:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 45414 extra nodes, 0 pruned nodes, max_depth=35
[21:54:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14704 extra nodes, 0 pruned nodes, max_depth=32
[21:55:02] INFO: /workspace/src/tree/updater_prune.cc:74: tree prunin

In [ ]:
y_pred=knn_inlier.predict(DtX)
print("Prediction End--")

In [ ]:
# Inlier statistics
from sklearn.metrics import accuracy_score
accuracy =  accuracy_score(y_test, y_pred) * 100
print(f"Accuracy a testhalmazra nézve: {accuracy}")
y_pred2=knn.predict(dX1)
accuracy =  accuracy_score(y_pred2, y_train_inliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



In [ ]:
dX1_ol=pd.DataFrame (X_train_outliers)
dY1_ol=pd.DataFrame (y_train_outliers)
DtX=pd.DataFrame(X_test)

In [ ]:
# készítsünk modell az outlierekre is
# KNN outlier modell

from xgboost import XGBClassifier  
knn_outlier = XGBClassifier(verbosity=3,max_depth = 35)

In [ ]:
print("Fit Start--")
knn_outlier.fit(dX1_ol, dY1_ol)
print("Fit End--")

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_ol=knn_outlier.predict(dX1_ol)
accuracy =  accuracy_score(y_pred_ol, y_train_outliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



In [ ]:
# megvan a két modell, inlierekre és oulierekre


In [ ]:
#az eredeti adatok itt vannak
df_testvalues

In [ ]:
# szét kellene kapni inlierekre és outlierekre a korábban betanított modellel.
test_outliers=clf.predict(df_testvalues)
outlierStatistic(test_outliers)

In [ ]:
#szedjük szét inlier és outlier listákra


X_test_inliers=[X_train[inx] for inx,i in enumerate(test_outliers) if i==1 ]
X_test_outliers=[X_train[index] for index,i in enumerate(test_outliers) if i==-1 ]

In [ ]:
df_X_test_inliers=pd.DataFrame (X_test_inliers)
df_X_test_outliers=pd.DataFrame (X_test_outliers)

In [ ]:
#predikáljuk az értékeket
#van betanított inlier predikátorunk
y_inlier_predikt=knn_inlier.predict(df_X_test_inliers)
y_outlier_predikt=knn_outlier.predict(df_X_test_outliers)

In [ ]:
df_test_buildings=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_a.csv")
df_buildings=df_test_buildings[["building_id"]]

In [ ]:
building_id=df_buildings[["building_id"]].values

In [ ]:
building_id

In [ ]:
building=[]
damage=[]
opi=0 # outlier predikt index
ipi=0 # inlier predikt index
bid=0
#össze kellene rakni az eredeti listát
for idx,i in enumerate(test_outliers):
    building.append(building_id[idx][0])  # idx-el kell hivatkozni 
    if i==-1:
        damage.append(y_outlier_predikt[opi])
        opi+=1
    else:
        damage.append(y_inlier_predikt[ipi])    
        ipi+=1

In [ ]:
print(len(building))

In [ ]:

outdf=pd.DataFrame(data={"damage_grade":damage} ,index=building)
outdf.index.name="building_id"


In [ ]:
outdf.head()

In [ ]:
basedir="/content/drive/My Drive/001_AI"
outdf.to_csv(basedir+"/_EarthQuake/submission_6_outlier.csv")

In [ ]:
!head "/content/drive/My Drive/001_AI/_EarthQuake/submission_6_outlier.csv"

In [ ]:
!head "/content/drive/My Drive/001_AI/_EarthQuake/test_a.csv"